# **<u>Targetmart2 DataLoader</u>**

## **Loading Data into dim.Customers**

In [1]:
GO

INSERT INTO dim.Customers(CustomerID, Customer, City, Country, LoadDate, SourceCountry)
	SELECT nwc.CustomerID
		  ,nwc.CompanyName 
		  ,nwc.City
		  ,nwc.Country
		  ,CAST(getdate() as DATE)
		  ,concat('Country = ', nwc.Country) 
	FROM Northwind_2023.dbo.Customers nwc
	WHERE nwc.CustomerID not in (SELECT CustomerID FROM dim.Customers)

;
GO

(0 rows affected)

Total execution time: 00:00:00.011

In [3]:
Select Top 5 * from dim.Customers

(5 rows affected)

Total execution time: 00:00:00.036

pkCustomer,CustomerID,Customer,City,Country,LoadDate,SourceCountry
1000,ALFKI,Alfreds Futterkiste,Berlin,Germany,2023-11-30,Country = Germany
1001,ANATR,Ana Trujillo Emparedados y helados,México D.F.,Mexico,2023-11-30,Country = Mexico
1002,ANTON,Antonio Moreno Taquería,México D.F.,Mexico,2023-11-30,Country = Mexico
1003,AROUT,Around the Horn,London,UK,2023-11-30,Country = UK
1004,BERGS,Berglunds snabbköp,Luleå,Sweden,2023-11-30,Country = Sweden


```
First script inserts data into the 'dim.Customers' table from the 'Northwind_2023.dbo.Customers' table. It selects specific columns like 'CustomerID,' 'CompanyName,' 'City,' 'Country,' and appends the current date as 'LoadDate' and a concatenation of 'Country = ' and 'nwc.Country' as 'SourceCountry.' The insertion is based on the condition that the 'CustomerID' does not already exist in 'dim.Customers'.

Second Script verifies the data has been loaded successfully.
```

## **Loading Data Into dim.Calendar**

In [5]:
IF (SELECT count(*) FROM dim.Calendar) = 0
BEGIN

DECLARE @StartDate DATE = '2020-01-01'
DECLARE @EndDate DATE = DATEADD(year, 0, GETDATE())
DECLARE @Date DATE = @StartDate
DECLARE @DayID INT = (datepart(year, @StartDate)-1900)*1000 + datepart(dy, @StartDate)
;


WHILE @Date <= @EndDate
	BEGIN
		INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Quarter, Qtr, Month,  MonthName, MonthShort, Week, Day, DayName, DayShort, IsWeekday, Weekday)
		VALUES (
			@DayID,
			@Date,
			YEAR(@Date),
			DATEPART(QUARTER, @Date),
			CASE WHEN DATEPART(QUARTER, @Date) IN (1) THEN '1st'
				 WHEN DATEPART(QUARTER, @Date) IN (2) THEN '2nd'
				 WHEN DATEPART(QUARTER, @Date) IN (3) THEN '3rd'
				 WHEN DATEPART(QUARTER, @Date) IN (4) THEN '4th'
				 ELSE '5th'
				 END,
			MONTH(@Date),
			DATENAME(MONTH, @Date),
			LEFT(DATENAME(MONTH, @Date),3),
			DATEPART(WEEK, @Date),
			DAY(@Date),
			DATENAME(WEEKDAY, @Date),
			LEFT(DATENAME(WEEKDAY, @Date),3),
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 0 ELSE 1 END, -- Set IsWeekday to 0 for Saturday (1) and Sunday (7), and 1 for weekdays
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 'No' ELSE 'Yes' END
	 )

	
		SET @Date = DATEADD(DAY, 1, @Date)
		SET @DayID = @DayID + 1
	END
END
GO

Commands completed successfully.

Total execution time: 00:00:00.004

In [7]:
SELECT top 5 * from dim.Calendar

(5 rows affected)

Total execution time: 00:00:00.017

pkCalendar,DateValue,Year,Quarter,Qtr,Month,MonthName,MonthShort,Week,Day,DayName,DayShort,IsWeekday,Weekday
120001,2020-01-01,2020,1,1st,1,January,Jan,1,1,Wednesday,Wed,1,Yes
120002,2020-01-02,2020,1,1st,1,January,Jan,1,2,Thursday,Thu,1,Yes
120003,2020-01-03,2020,1,1st,1,January,Jan,1,3,Friday,Fri,1,Yes
120004,2020-01-04,2020,1,1st,1,January,Jan,1,4,Saturday,Sat,0,No
120005,2020-01-05,2020,1,1st,1,January,Jan,2,5,Sunday,Sun,0,No


```
First script populates the 'dim.Calendar' table if it is initially empty. It declares variables for start and end dates, sets up a loop to insert date-related information into the calendar, such as 'pkCalendar,' 'DateValue,' 'Year,' 'Quarter,' 'Qtr,' 'Month,' 'MonthName,' 'MonthShort,' 'Week,' 'Day,' 'DayName,' 'DayShort,' 'IsWeekday,' and 'Weekday.' The loop runs from '2020-01-01' to the current date, ensuring weekdays are marked correctly, and 'IsWeekday' is set to 0 for Saturdays and Sundays, and 1 for weekdays.

Second Script Verifies the data has been loaded successfully.
```

## **Loading Data Into dim.Shipper**

In [11]:
INSERT INTO dim.Shipper(pkShipId, Shipper, Phone)
SELECT s_sh.ShipperID
      ,s_sh.CompanyName
      ,s_sh.Phone
FROM Northwind_2023.dbo.Shippers s_sh
WHERE s_sh.ShipperID not in (SELECT pkShipId FROM dim.Shipper)
;
GO

(0 rows affected)

Total execution time: 00:00:00.003

In [12]:
SELECT  * from dim.Shipper

(3 rows affected)

Total execution time: 00:00:00.005

pkShipId,Shipper,Phone
1,Speedy Express,(503) 555-9831
2,United Package,(503) 555-3199
3,Federal Shipping,(503) 555-9931


```
First script inserts data into the 'dim.Shipper' table from the 'Northwind_2023.dbo.Shippers' table. It selects specific columns like 'ShipperID,' 'CompanyName,' and 'Phone' and inserts records where the 'ShipperID' does not already exist in 'dim.Shipper.' This helps maintain an updated and comprehensive list of shippers in the dimensional model.

Second Query Verifies the data has been loaded successfully.
```

## **Loading Data Into dim.Products**

In [13]:
INSERT INTO dim.Products(pkProdId, ProductId, Product, ProductCategory, CategoryDesc, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued, ReorderFlag)
SELECT prod.ProductID + 10000 as 'pkProdId'
      , prod.ProductID as 'ProductId'
      , prod.ProductName as 'Product'
	  , cat.CategoryName as 'ProductCategory'
	  , cat.[Description] as 'CategoryDesc'
	  , prod.UnitPrice
	  , prod.UnitsInStock
	  , prod.UnitsOnOrder
	  , prod.ReorderLevel
	  , prod.Discontinued
	  , CASE 
			WHEN prod.ReorderLevel >= (prod.UnitsInStock + prod.UnitsOnOrder)
			     AND prod.Discontinued = 0
			 THEN 'Yes'
	      ELSE 'No'
	    END as 'ReorderFlag'
FROM Northwind_2023.dbo.Products prod
	INNER JOIN Northwind_2023.dbo.Categories cat
	ON prod.CategoryID = cat.CategoryID
WHERE prod.ProductID not in (SELECT ProductId FROM dim.Products)
;



GO

(0 rows affected)

Total execution time: 00:00:00.013

In [14]:
Select top 5 * from dim.Products

(5 rows affected)

Total execution time: 00:00:00.017

pkProdId,ProductId,Product,ProductCategory,CategoryDesc,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,ReorderFlag
10001,1,Chai,Beverages,"Soft drinks, coffees, teas, beers, and ales",18.00,39,0,10,0,No
10002,2,Chang,Beverages,"Soft drinks, coffees, teas, beers, and ales",19.00,17,40,25,0,No
10003,3,Aniseed Syrup,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",10.00,13,70,25,0,No
10004,4,Chef Anton's Cajun Seasoning,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",22.00,53,0,0,0,No
10005,5,Chef Anton's Gumbo Mix,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",21.35,0,0,0,1,No


```
First script inserts data into the 'dim.Products' table from the 'Northwind_2023.dbo.Products' and 'Northwind_2023.dbo.Categories' tables. It selects specific columns like 'ProductID,' 'ProductName,' 'CategoryName,' and calculates additional values for 'pkProdId' and 'ReorderFlag.' The insertion is based on the condition that the 'ProductID' does not already exist in 'dim.Products.' The script enhances the dimensional model with product information and a calculated flag indicating whether a product needs to be reordered.

Second Query Verifies the data uploded successfully.
```

Loading Data into dim.Employees

In [15]:
INSERT INTO dim.Employees(EmployeeID, Employee, Title, BirthDate, HireDate, City, Country, ReportsTo)
SELECT sEmp.EmployeeID
	  ,concat( sEmp.TitleOfCourtesy, ' ', sEmp.FirstName, ', ', sEmp.LastName)
      ,sEmp.Title
      ,sEmp.BirthDate
      ,sEmp.HireDate
      ,sEmp.City
      ,sEmp.Country
      ,sEmp.ReportsTo
FROM Northwind_2023.dbo.Employees sEmp
WHERE sEmp.EmployeeID not in (SELECT EmployeeID FROM dim.Employees)

(0 rows affected)

Total execution time: 00:00:00.009

In [16]:
Select Top 5 * from dim.Employees

(5 rows affected)

Total execution time: 00:00:00.010

EmployeeID,Employee,Title,BirthDate,HireDate,City,Country,ReportsTo
1,"Ms. Nancy, Davolio",Sales Representative,1973-12-08 00:00:00.000,2017-05-01 00:00:00.000,Seattle,USA,2
2,"Dr. Andrew, Fuller","Vice President, Sales",1977-02-19 00:00:00.000,2017-08-14 00:00:00.000,Tacoma,USA,NULL
3,"Ms. Janet, Leverling",Sales Representative,1988-08-30 00:00:00.000,2017-04-01 00:00:00.000,Kirkland,USA,2
4,"Mrs. Margaret, Peacock",Sales Representative,1962-09-19 00:00:00.000,2018-05-03 00:00:00.000,Redmond,USA,2
5,"Mr. Steven, Buchanan",Sales Manager,1980-03-04 00:00:00.000,2018-10-17 00:00:00.000,London,UK,2


```
First script inserts data into the 'dim.Employees' table from the 'Northwind_2023.dbo.Employees' table. It selects specific columns like 'EmployeeID,' 'TitleOfCourtesy,' 'FirstName,' 'LastName,' 'Title,' 'BirthDate,' 'HireDate,' 'City,' 'Country,' and 'ReportsTo.' The insertion is based on the condition that the 'EmployeeID' does not already exist in 'dim.Employees.' This enhances the dimensional model with employee information for analysis and reporting.

Second query Verifies the data uploaded successfully.
```

## **Loading Data Into f.OrderPerf**

In [18]:

TRUNCATE TABLE TargetMart2.f.OrderPerf
GO

INSERT INTO TargetMart2.f.OrderPerf(OrderID, fkCalendar, fkCustomer, fkShipper, fkEmployee, Freight, DaysToShip, DaysTilRequired, OrderCount)
SELECT sO.OrderID
	  ,tC.pkCalendar as fkCalendar
	  ,tCus.pkCustomer as fkCustomer
	  ,sO.ShipVia as fkShipper
	  ,sO.EmployeeID as fkEmployee
	  ,sO.Freight
	  ,DATEDIFF ( day , sO.OrderDate , sO.ShippedDate ) as 'DaysToShip'
	  ,DATEDIFF ( day , sO.OrderDate , sO.RequireDate ) as 'DaysTilRequired'
	  , 1 as 'OrderCount'
FROM Northwind_2023.dbo.Orders sO
	INNER JOIN TargetMart2.dim.Calendar tC
	ON cast(so.OrderDate as DATE) = tc.DateValue
	INNER JOIN TargetMart2.dim.Customers tCus
	ON sO.CustomerID = tCus.CustomerID
; 

Commands completed successfully.

(830 rows affected)

Total execution time: 00:00:00.036

In [19]:
Select Top 5 * from f.OrderPerf

(5 rows affected)

Total execution time: 00:00:00.008

OrderID,fkCalendar,fkCustomer,fkShipper,fkEmployee,Freight,DaysToShip,DaysTilRequired,OrderCount
10248,120551,1084,3,5,32.38,12,28,1
10249,120552,1078,1,6,11.61,5,42,1
10250,120555,1033,2,4,65.83,4,28,1
10251,120555,1083,1,3,41.34,7,28,1
10252,120556,1075,2,4,51.30,2,28,1


```
First script truncates the 'TargetMart2.f.OrderPerf' table and then inserts data into it from the 'Northwind_2023.dbo.Orders' table. It selects specific columns like 'OrderID,' 'ShipVia,' 'EmployeeID,' 'Freight,' and calculates values like 'DaysToShip' and 'DaysTilRequired.' The insertion is based on joining with the 'TargetMart2.dim.Calendar' and 'TargetMart2.dim.Customers' tables, enhancing the data warehouse with order performance information.

Second query verifies the data loaded successfully.
```

## **Loading data into f.ProductPerf**

In [20]:
TRUNCATE TABLE TargetMart2.f.ProductPerf
GO

INSERT INTO TargetMart2.f.ProductPerf(OrderID, fkOrderDate, fkProductID, fkCustomerID, UnitPrice, Quantity, Discount, DiscFlag, LineTotal)
SELECT sDet.OrderID
      ,tCal.pkCalendar as 'OrderDate'
      ,sDet.ProductID + 10000 as 'ProductID'
--	  ,sOrd.CustomerID
	  ,tCust.pkCustomer
      ,sDet.UnitPrice
      ,sDet.Quantity
      ,sDet.Discount
	  ,CASE 
		 WHEN sDet.Discount = 0 THEN 0
		 ELSE 1
	   END as 'DiscFlag'
	  ,CAST(((1 - sDet.Discount) * sDet.UnitPrice) * sDet.Quantity as MONEY) as 'LineTotal'
FROM Northwind_2023.dbo.OrderDetails sDet
	INNER JOIN Northwind_2023.dbo.Orders sOrd
	ON sDet.OrderID = sOrd.OrderID
	INNER JOIN dim.Calendar tCal
	ON CAST(sOrd.OrderDate as DATE) = tCal.DateValue
	INNER JOIN dim.Customers tCust
	ON sOrd.CustomerID = tCust.CustomerID
;


Commands completed successfully.

(2155 rows affected)

Total execution time: 00:00:00.058

In [21]:
Select top 5 * from f.ProductPerf

(5 rows affected)

Total execution time: 00:00:00.014

OrderID,fkOrderDate,fkProductID,fkCustomerID,UnitPrice,Quantity,Discount,DiscFlag,LineTotal
10248,120551,10011,1084,14.00,12,0,0,168.00
10248,120551,10042,1084,9.80,10,0,0,98.00
10248,120551,10072,1084,34.80,5,0,0,174.00
10249,120552,10014,1078,18.60,9,0,0,167.40
10249,120552,10051,1078,42.40,40,0,0,1696.00


```
First script first truncates the 'TargetMart2.f.ProductPerf' table and then inserts data into it from the 'Northwind_2023.dbo.OrderDetails' table. It selects specific columns like 'OrderID,' 'ProductID,' 'UnitPrice,' 'Quantity,' 'Discount,' and calculates values like 'DiscFlag' and 'LineTotal.' The insertion is based on joining with 'Northwind_2023.dbo.Orders,' 'dim.Calendar,' and 'dim.Customers' tables, enhancing the data warehouse with product performance information.

Second query Verifies the data loaded successfully.
```